In [13]:
#import projectionLineOfSight as ao
import numpy as np
import math
import random as ran

# 1) Create ToyModel

In [ ]:
#Tutte variabili da cancellare
#real*8 Ldisc,L1,L2,L3,pi,iota,mass,x(0:1000000),y(0:1000000)
# $     ,z(0:1000000),vx(1000000),vy(1000000),vz(1000000),gamma,t ,x1
# $     ,x2,y1,y2,z1,z2,rz,ry,rx,r(1000000),r2 ,vxRd(1000000)
# $     ,vyRd(1000000),vzRd(1000000),vRad ,vxRt(1000000)
# $     ,vyRt(1000000),vzRt(1000000),vRot,vradial ,vrotational
# $     ,Vtot(1000000) ,dr,omega,rmin,rmax,alpha,A_vr,B_vr,A_vc,B_vc
# $     ,xfact, xobs_k(10000),yobs_k(10000),vobs_k(10000),xobs_k_max
# $     ,xobs_k_min,xobs_o(10000),yobs_o(10000),vobs_o(10000)
# $     ,xobs_o_max ,xobs_o_min, a_maj, a_min,vobs_k_max,vobs_k_min
# $     ,dx,vbin,vrotation,x_max,x_min,vx_ave(0:1000),x_ave(0:1000)
# $     ,vx_max,x8

#  integer iN,Np,i,j,imax,iseed,n_k,n_o,k,ks,kk,l,Nl,Nbin,nx
# $     ,nx_ave(1000) 



#************** CONSTANTS ***************
      
pi=math.pi
mass=0.0001 

#Definisce dei file con label 10,12,13,11. A fine del vecchio programma venivano salvati questi 4 file
#Noi siamo interessati al 10

#open(10,file='toymodel.dat',status='unknown')
#open(12,file='toymodel1-rand.dat',status='unknown')
#open(13,file='toymodel2-rand.dat',status='unknown')
#open(11,file='toymodel_prof.dat',status='unknown')

x8=1.000000000000
Np=500000
Ldisc=600.
print('Size  disk=%f' %Ldisc)

iota=0. 
print('iota=%f' %iota)

L1=1.5*Ldisc
L2=1.*Ldisc
L3=Ldisc/10.
print('L1,L2,L3=%f%f%f' %(L1,L2,L3))

#print('L1,L2,L3 [semi axis]=',L1,L2,L3) #---> non capisco
      
print('Np= %i' %Np)
      
      
#*************************************************
x=y=z=r=vxRd=vyRd=vzRd=vxRt=vyRt=vzRt=vx=vy=vz=vtot = np.zeros((Np))

x[0]=0.
y[0]=0.
z[0]=0.
       
#**************** disk ***************************

#j=0                                                 A COSA SERVE QUESTO j?

#Per ogni punto
for i in range(1,Np):
    
    #x1, y1 e z1 devono rispettare delle condizioni; creo un ciclo while che si ferma quando vengono rispettate
    #Per entrare nel while setto le 3 variabili (ed r2) in modo tale da entrare nel ciclo
    x1 = L1 + 1
    y1 = L2 + 1
    z1 = L3 + 1
    r2 = 2.
    
    while((x1 < -L1) and (x1 > L1) and (y1 < -L2) and (y1 > L2) and (z1 < -L3) and (z1 > L3) and (r2 > 1)):
        
        x1=2.*L1*(ran.uniform(0,1)-0.5)
        y1=2.*L2*(ran.uniform(0,1)-0.5)
        z1=2.*L3*(ran.uniform(0,1)-0.5)
        
        #Non inserisco rz perché non serve per rispettare le 4 condizioni 
        rx=x1-x[0]
        ry=y1-y[0]

        r2 = np.sqrt(rx*rx/(L1)**2.+ry*ry/(L2)**2)

    #Uscito dal while faccio tutte le assegnazioni
    x[i]=x1
    y[i]=y1
    z[i]=z1
    
    rx=x[i]-x[0]
    ry=y[i]-y[0]
    rz=z[i]-z[0]
    
    r[i] = np.sqrt(rx*rx+ry*ry+rz*rz)
    alpha = 180./pi*np.arctan(np.abs(ry)/np.abs(rx))
    
    
    if(r[i] <= 200):
        vradial = 0.
        vrotation = 100.

    else: #---> PARTE DA MODIFICARE
        vradial = 0.4*r[i]
        vrotation = 000./(r[i]/200.)

    vxRd[i] = abs(vradial)*np.abs(rx)/r[i] #radial component 
    vyRd[i] = abs(vradial)*np.abs(ry)/r[i]
    vzRd[i] = abs(vradial)*np.abs(rz)/r[i]

    if(rx < 0):
        vxRd[i] = -vxRd[i]
    if(ry < 0):
        vyRd[i] = -vyRd[i]
    if(rz < 0):
        vzRd[i] = -vzRd[i]


#       vRad=sqrt(vxRd(i)*vxRd(i)+vyRd(i)*vyRd(i)+vzRd(i)*vzRd(i))


#       vxRt(i)=+(ry)*vrotation/r(i) !clockwise --> anticlockwise invert signs
#       vyRt(i)=-(rx)*vrotation/r(i)
#       vzRt(i)=0.

    vxRt[i] = -(ry)*vrotation/r[i] #anticlockwise --> clockwise invert signs
    vyRt[i] = +(rx)*vrotation/r[i]
    vzRt[i] = 0.

    vRot = np.sqrt(vxRt[i]*vxRt[i]+vyRt[i]*vyRt[i]+vzRt[i]*vzRt[i])

    vx[i] = vxRt[i]+vxRd[i]
    vy[i] = vyRt[i]+vyRd[i]
    vz[i] = vzRt[i]+vzRd[i]

    vtot[i] = np.sqrt((vx[i]**2.+(vy[i]**2.+(vz[i]**2.))))

    #write(10,*)x(i),y(i),z(i),vx(i),vy(i),vz(i),x8 ---> Questo è il file che poi, prima di questo codice,
                                                        #assegnavamo al codice python che proietta sulla
                                                        #linea di vista

# 2) Proiezione lungo la linea di vista (è già file python, quindi dovrebbe essere semplice)